# Gym ENv
https://gym-trading-env.readthedocs.io/en/latest/

In [1]:
#import gym_trading_env

## Tutorial
https://gym-trading-env.readthedocs.io/en/latest/rl_tutorial.html

### Market data

In [2]:
import pandas as pd
# Available in the github repo : examples/data/BTC_USD-Hourly.csv
url = "https://raw.githubusercontent.com/ClementPerroud/Gym-Trading-Env/main/examples/data/BTC_USD-Hourly.csv"
df = pd.read_csv(url, parse_dates=["date"], index_col= "date")
df.sort_index(inplace= True)
df.dropna(inplace= True)
df.drop_duplicates(inplace=True)

In [3]:
#from gym_trading_env.downloader import download
import datetime
import pandas as pd

# Download BTC/USDT historical data from Binance and stores it to directory ./data/binance-BTCUSDT-1h.pkl
if False:
    download(exchange_names = ["binance"],
        symbols= ["BTC/USDT"],
        timeframe= "1h",
        dir = "data",
        since= datetime.datetime(year= 2020, month= 1, day=1),
    )
    # Import your fresh data
    df = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")

In [4]:
df

,unix,symbol,open,high,low,close,volume,Volume USD
date,,,,,,,,
2018-05-15 06:00:00,1526364000,BTC/USD,8733.86,8796.68,8707.28,8740.99,4.906603e+06,5.599300e+02
2018-05-15 07:00:00,1526367600,BTC/USD,8740.99,8766.00,8721.11,8739.00,2.390399e+06,2.735800e+02
2018-05-15 08:00:00,1526371200,BTC/USD,8739.00,8750.27,8660.53,8728.49,7.986063e+06,9.177900e+02
2018-05-15 09:00:00,1526374800,BTC/USD,8728.49,8754.40,8701.35,8708.32,1.593992e+06,1.826200e+02
2018-05-15 10:00:00,1526378400,BTC/USD,8708.32,8865.00,8695.11,8795.90,1.110127e+07,1.260690e+03
...,...,...,...,...,...,...,...,...
2022-02-28 20:00:00,1646078400,BTC/USD,41361.99,41971.00,41284.11,41914.97,2.471517e+02,1.035935e+07
2022-02-28 21:00:00,1646082000,BTC/USD,41917.09,41917.09,41542.60,41659.53,6.975168e+01,2.905822e+06
2022-02-28 22:00:00,1646085600,BTC/USD,41657.23,44256.08,41650.29,42907.32,5.275406e+02,2.263535e+07


### Create Features
Your RL-agent will need inputs. It is your job to make sure it has everything it needs.
The environment will recognize as inputs every column that contains the keyword ‘feature’ in its name.

In [5]:
# df is a DataFrame with columns : "open", "high", "low", "close", "Volume USD"

# Create the feature : ( close[t] - close[t-1] )/ close[t-1]
df["feature_close"] = df["close"].pct_change()

# Create the feature : open[t] / close[t]
df["feature_open"] = df["open"]/df["close"]

# Create the feature : high[t] / close[t]
df["feature_high"] = df["high"]/df["close"]

# Create the feature : low[t] / close[t]
df["feature_low"] = df["low"]/df["close"]

 # Create the feature : volume[t] / max(*volume[t-7*24:t+1])
df["feature_volume"] = df["Volume USD"] / df["Volume USD"].rolling(7*24).max()

df.dropna(inplace= True) # Clean again !
# Eatch step, the environment will return 5 inputs  : "feature_close", "feature_open", "feature_high", "feature_low", "feature_volume"



What is presented above are features called static features. Indeed, there are computed once before being used in the environement. But you also use dynamic features that are computed at each step of the environment. By default, the environment add 2 dynamics features. More information in the Feature page.

### Create the environment


In [6]:
import gymnasium as gym
#import gym_trading_env
# env = gym.make("TradingEnv",
#         name= "BTCUSD",
#         df = df, # Your dataset with your custom features
#         positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
#         trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
#         borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
#     )

In [7]:
from src.gym_trading_env.environments import TradingEnv

env = TradingEnv(
        name= "BTCUSD",
        df = df, # Your dataset with your custom features
        positions = [ -1, 0, 1], # -1 (=SHORT), 0(=OUT), +1 (=LONG)
        trading_fees = 0.01/100, # 0.01% per stock buy / sell (Binance fees)
        borrow_interest_rate= 0.0003/100, # 0.0003% per timestep (one timestep = 1h here)
    )

### Run the environment

In [14]:
# Run an episode until it ends :
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)
    # print(position_index)
    # To render
    if done:
        print(env.options)
        # At the end of the episode you want to render
        env.save_for_render(dir = "render_logs")
        

Market Return : 423.10%   |   Portfolio Return : -88.62%   |   


In [12]:
env.options

shape: (0, 0)
┌┐
╞╡
└┘

In [10]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

UserWarning: [33mWARN: Overriding environment TradingEnv already in registry.[0m